# Data Anonymization techniques

- Masking: Hiding part of sensitive data (e.g., email, SSN, phone).
- Hashing: Replacing data with irreversible hashed values (ideal for IDs).
- Tokenization/Pseudonymization: Replacing sensitive values with random tokens that can be reversed later using a lookup table.
- Redaction/Removal: Completely removing sensitive columns or rows.
- Generalization:  Reducing precision to hide exact values.
- Noise Addition: Altering values slightly to hide exact data (esp. for analytics).
- NER - Named entity recognition

[microsoft presidio](https://microsoft.github.io/presidio/)

In [ ]:
# setup


In [ ]:
# load the user data


In [ ]:
# masking email 


In [ ]:
# hashing SSN credit card




In [ ]:
# generaliation



In [ ]:
# noise addition to salary




In [ ]:
# save the changes


In [ ]:
! pip install presidio-analyzer presidio_structured presidio-anonymizer faker pandas && python -m spacy download en_core_web_lg


In [ ]:
# NER